In [ ]:
from google.colab import drive

def mount_gdrive(gdrive_folder):
    # Mount Google Drive and set the mount point to the specified folder
    drive.mount('/gdrive')

    # Change the current working directory to the project folder
    project_folder = f'/gdrive/{gdrive_folder}'
    %cd $project_folder

# Specify your Google Drive folder here
your_gdrive_folder = 'MyDrive/---'

# Mount Google Drive and set the current working directory
mount_gdrive(your_gdrive_folder)


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/BlackCoffer_Task


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_page_content(url, url_id):
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

    try:
        response = requests.get(url, headers=header)
        response.raise_for_status()  # Raise an HTTPError for bad responses
    except requests.exceptions.RequestException as e:
        print(f"Error getting response for {url_id}: {e}")
        return None

    try:
        soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Error creating BeautifulSoup object for {url_id}: {e}")
        return None

    return soup

def main():
    # Read the url file into the pandas object
    try:
        df = pd.read_excel('Input.xlsx')
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    # Loop through each row in the df
    for index, row in df.iterrows():
        url = row['URL']
        url_id = row['URL_ID']

        # Get page content
        soup = get_page_content(url, url_id)
        if soup is None:
            continue

        # Find title
        try:
            title = soup.find('h1').get_text()
        except AttributeError as e:
            print(f"Error getting title for {url_id}: {e}")
            continue

        # Find text
        article = ""
        try:
            for p in soup.find_all('p'):
                article += p.get_text()
        except Exception as e:
            print(f"Error getting text for {url_id}: {e}")

        # Write title and text to the file
        file_name = f'/gdrive/MyDrive/BlackCoffer_Task/TitleText/{url_id}.txt'
        try:
            with open(file_name, 'w', encoding='utf-8') as file:
                file.write(title + '\n' + article)
        except Exception as e:
            print(f"Error writing to file {file_name}: {e}")

if __name__ == "__main__":
    main()


Error getting response for blackassign0036: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error getting response for blackassign0049: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


In [ ]:
# Directories
text_dir = "/gdrive/MyDrive/---/TitleText"
stopwords_dir = "/gdrive/MyDrive/---/StopWords"
sentment_dir = "/gdrive/MyDrive/---/MasterDictionary"

# load all stop wors from the stopwords directory and store in the set variable
stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dir):
  with open(os.path.join(text_dir,text_file),'r') as f:
    text = f.read()
#tokenize the given text file
    words = word_tokenize(text)
# remove the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
# add each filtered tokens of each file into a list
    docs.append(filtered_text)



# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      pos.update(f.read().splitlines())
  else:
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      neg.update(f.read().splitlines())

# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos])
  Negative_words.append([word for word in docs[i] if word.lower() in neg])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))


In [ ]:
import os
import re
from nltk.corpus import stopwords

text_dir = "/gdrive/MyDrive/---/TitleText"
stopwords = set(stopwords.words('english'))

def measure_complexity(file_path):
    with open(file_path, 'r') as f:
        text = f.read()

    # Remove punctuations
    text = re.sub(r'[^\w\s.]', '', text)

    # Split the text into sentences
    sentences = text.split('.')

    # Total number of sentences in a file
    num_sentences = len(sentences)

    # Total words in the file
    words = [word for word in text.split() if word.lower() not in stopwords]
    num_words = len(words)

    # Complex words (syllable count greater than 2)
    complex_words = [word for word in words if sum(1 for letter in word if letter.lower() in 'aeiou') > 2]

    # Syllable Count Per Word
    syllable_count = sum(sum(1 for letter in word if letter.lower() in 'aeiou') for word in words)

    # Average sentence length
    avg_sentence_length = num_words / num_sentences

    # Percentage of Complex words
    percentage_of_complex_words = len(complex_words) / num_words

    # Fog Index
    fog_index = 0.4 * (avg_sentence_length + percentage_of_complex_words)

    # Average syllable count per word
    avg_syllable_word_count = syllable_count / num_words

    return avg_sentence_length, percentage_of_complex_words, fog_index, len(complex_words), avg_syllable_word_count

# Measure complexity for each file in text_dir
avg_sentence_length = []
percentage_of_complex_words = []
fog_index = []
complex_word_count = []
avg_syllable_word_count = []

for file_name in os.listdir(text_dir):
    file_path = os.path.join(text_dir, file_name)
    x, y, z, a, b = measure_complexity(file_path)
    avg_sentence_length.append(x)
    percentage_of_complex_words.append(y)
    fog_index.append(z)
    complex_word_count.append(a)
    avg_syllable_word_count.append(b)


In [ ]:
import os
import re
from nltk.corpus import stopwords

text_dir = "/gdrive/MyDrive/---/TitleText"
stopwords = set(stopwords.words('english'))

def cleaned_words(file_path):
    with open(file_path, 'r') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]', '', text)
        words = [word for word in text.split() if word.lower() not in stopwords]
        total_word_length = sum(len(word) for word in words)
        average_word_length = total_word_length / len(words)
    return len(words), average_word_length

def count_personal_pronouns(file_path):
    with open(file_path, 'r') as f:
        text = f.read()
        personal_pronouns = ["I", "we", "my", "ours", "us"]
        count = sum(len(re.findall(r"\b" + pronoun + r"\b", text)) for pronoun in personal_pronouns)
    return count

# Word Count and Average Word Length
word_count = []
average_word_length = []
for file_name in os.listdir(text_dir):
    file_path = os.path.join(text_dir, file_name)
    x, y = cleaned_words(file_path)
    word_count.append(x)
    average_word_length.append(y)

# Count Personal Pronouns
pp_count = []
for file_name in os.listdir(text_dir):
    file_path = os.path.join(text_dir, file_name)
    x = count_personal_pronouns(file_path)
    pp_count.append(x)

In [ ]:
import pandas as pd

# Read the output data structure from the Excel file
output_df = pd.read_excel('Output Data Structure.xlsx')

# URL_ID 36 and 49 do not exist (i.e., page does not exist and throws a 404 error)
# Drop these rows from the table
output_df.drop([36, 49], axis=0, inplace=True)

# Define the required parameters
variables = [
    positive_score,
    negative_score,
    polarity_score,
    subjectivity_score,
    avg_sentence_length,
    percentage_of_complex_words,
    fog_index,
    avg_sentence_length,
    complex_word_count,
    word_count,
    avg_syllable_word_count,
    pp_count,
    average_word_length
]

# Write the values to the dataframe
for i, var in enumerate(variables):
    output_df.iloc[:, i + 2] = var

# Save the dataframe to a CSV file
output_df.to_csv('Output_Data.csv', index=False)


<ipython-input-18-c702e75f6d38>:29: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  output_df.iloc[:, i + 2] = var
